# Python Example 3: Simulating SKS splitting

In this example we generate P receiver functions for a model that includes a lower-crustal anisotropic layer. This example follows that of Figure 2 in Porter et al. (2011). 

Start by importing the necessary modules

In [1]:
import numpy as np
from obspy.core import Stream
from obspy.signal.rotate import rotate_ne_rt
from telewavesim import plane as pw
from telewavesim import utils as ut
from telewavesim import wiggle as wg
from telewavesim import conf as cf

Select the model file:

In [2]:
modfile = '../../models/model_SKS.txt'

Select the type of incident wave - options are 'P', 'SV', 'SH', or 'Si', which is an isotropic S-wave source. SKS waves are SV polarized, so select `'SV'`

In [3]:
cf.wvtype = 'SV'

Next we use global variables to define the desired time series

In [4]:
cf.nt = 3000 # Number of samples
cf.dt = 0.05 # sampling rate in seconds

Now specify the parameters of the incident wavefield in terms of a horizontal slowness and back-azimuth. In this example the slowness won't change, so we can pass it as a global variable now. The back-azimuth will range from 0 to 360 degrees with a 10-degree increment, so we define a `np.ndarray` for this variable and do not yet pass it as a global variable.

In [5]:
cf.slow = 0.04 # Horizontal slowness (or ray parameter) in s/km 
baz = np.arange(0., 360., 10.)

Read the model parameters from the model file and set global variables (hidden in the function). Up to here, the steps could have been performed in no particular order, except the name of the file that needs to be defined before the call to `read_model()`

In [6]:
ut.read_model(modfile)

The following step is useful for plotting the plane wave seismograms with respect to the predicted propagation time through the model (from lowermost interface up to the seafloor). Since we are doing SKS wave modeling, we are interested in plane wave seismograms so let's calculate it.

In [7]:
t1 = ut.calc_ttime(cf.slow)
print('Predicted propagation time from model: {0:4.1f} sec'.format(t1))

Predicted propagation time from model: 21.6 sec


As we need to loop through back-azimuth values, we will initialize empty `Stream` objects to store the traces from the output of the main routine. 

In [8]:
trR = Stream(); trT = Stream()

Now the main loop over back-azimuths where all calculations are done - self explanatory

In [ ]:
# Loop over range of data 
for bb in baz:

    # Pass baz global variable
    cf.baz = bb

    # Calculate the plane wave seismograms
    trxyz = pw.plane_land()

    # Extract East, North and Vertical
    ntr = trxyz[0]
    etr = trxyz[1]
    ztr = trxyz[2]

    # Copy to radial and transverse
    rtr = ntr.copy()
    ttr = etr.copy()

    # Rotate to radial and transverse
    rtr.data, ttr.data = rotate_ne_rt(ntr.data, etr.data, bb)

    # Append to streams
    trR.append(rtr)
    trT.append(ttr)

The result of the previous loop is a set of radial and transverse seismograms. We simply filter the `Stream` objects using some frequency corners to simulate the frequency content of SKS waves

In [ ]:
# Set frequency corners in Hz
f1 = 0.01
f2 = 0.2

# Filter to get wave-like traces
trR.filter('bandpass',freqmin=f1, freqmax=f2, corners=2, zerophase=True)
trT.filter('bandpass',freqmin=f1, freqmax=f2, corners=2, zerophase=True)

In [ ]:
# Plot as wiggles
wg.pw_wiggles_baz(trR, trT, 'test', btyp='baz', scale=0.05, t1=t1, tmin=0., tmax=40, save=False, ftitle='_synt')

And Voilà! 

Can you find the fast direction and approximate delay time? Easy from the plot of radial components. Null measurements are also quite obvious from the absence of signal on the transverse component at back-azimtuths corresponding to the fast and slow axes. In this example the anisotropic layer is defined by a hexagonal symmetry with a fast axis, with 10% anisotropy. You can play with the model to have two anisotropic layers with different fast axis orientation, or use anisotropic elastic tensors determined in the lab (e.g., use `'ol'` (abbreviation for 'olivine') instead of `'tri'` in the definition of the layer in the model file). Enjoy!